# **ENV590.05 - Econ of Modern Power Systems - A5 - IPP producer reservoir operation model**

## Installing and Running Pyomo on Google Colab

To import/install a library that's not in Colaboratory by default, you can use !pip install. This needs to be done at the begining of you notebook. And you only need to run it once at the start of each Colab session.

Pyomo does not include any optimization solvers. Therefore, you will need to install third-party solvers to solve optimization models built with Pyomo. There are several solver options, for this class we will use glpk. We'll install glpk using apt-get.

In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
!apt-get install -y coinor-cbc

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

## Import Pyomo and solver

The first step in any Pyomo project is to import relevant components of the Pyomo library. This can be done with the following python statement 'from pyomo.environ import *'. \\

We use the * symbol to elimate the need of using the expression pyomo.environ every time we need to use a pyomo function. \\

You also need to load the solver. The Pyomo libary includes a SolverFactory() class used to specify a solver. Here we will use
**glpk** which works for linear problems, put **cbc** from coin OR could be used for nonlinear applications.

In [ ]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('cbc')

#opt = SolverFactory('appsi_highs')  # use the HiGHS solver via APPSI

## Creating Model with vectors and numpy

In [ ]:
#Importing numpy library to enable array use and pandas for data frames
import numpy as np
import pandas as pd

#Entering problem data
inflow = np.array([2, 2, 3, 4, 3, 2, 2, 1, 2, 3, 3, 2]) #UoW inflow
elect_price = np.array([1.6, 1.7, 1.8, 1.9, 2.0, 2.0, 2.0, 1.9, 1.8, 1.7, 1.6, 1.5]) #electricity price
irrig_price = np.array([1.0, 1.2, 1.8, 2.0, 2.2, 2.2, 2.5, 2.2, 1.8, 1.4, 1.1, 1.0]) #irrigation price
r0 = 5 #reservoir initial level
r12 = 5 #reservoir level in last month

T=range(12)  #twelve months of the year

In [ ]:
#Creating model
model = ConcreteModel()

#Index decision variables
model.e=Var(T,domain=NonNegativeReals) #dec var for electricity generation i.e. how much water for electricity generation == water released
model.i=Var(T,domain=NonNegativeReals) #dec var for irrigation i.e. how much water for irrigation
model.r=Var(T,domain=NonNegativeReals) #dec var tracking reservoir level from one month to the other

#Adding objective function
model.profit = Objective(
    expr = sum(elect_price[t]*model.e[t] for t in T) + sum(irrig_price[t]*model.i[t] for t in T),
    sense= maximize
    )

In [ ]:
#Adding constraints
model.constraints = ConstraintList()
for t in T:
    #Need at least 1 unit of water downstream
    model.constraints.add(1 <= model.e[t] - model.i[t])

    #reservoir water balance constraint
    if (t==0):
      model.constraints.add(model.r[t] == r0 + inflow[t] - model.e[t])
    else:
      model.constraints.add(model.r[t-1] + inflow[t] - model.e[t] == model.r[t])

    #reservoir level in the last month should be 5
    if (t==11):
      model.constraints.add(model.r[t] == r12)

    #simple bound e[t]: total outflow cannot exceed 7 units of water
    model.constraints.add(model.e[t] <= 7)

    #simple bound r[t]: reservoir level at any month cannot exceed 10 unit of water
    #model.constraints.add(0 <= model.r[t])
    model.constraints.add(model.r[t] <= 10)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
#Solving model
#opt.solve(model)

results = opt.solve(model, tee=True)

#print(results.solver.termination_condition)

#Print results
print('Profit = ', model.profit())
#print('\nDecision Variables')


Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpjt5facle.pyomo.lp -stat=1 -solve -solu /tmp/tmpjt5facle.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Statistics for presolved model


Problem has 22 rows, 33 columns (24 with objective) and 54 elements
There are 12 singletons with objective 
Column breakdown:
13 of type 0.0->inf, 20 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
10 of type E other, 0 of type G 0.0, 12 of type G 1.0, 
0 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of typ

In [ ]:
sys_op_sch = pd.DataFrame(np.zeros((12,4)))

for t in T:
  sys_op_sch.iloc[t,0] = t+1
  sys_op_sch.iloc[t,1] =  model.e[t].value
  sys_op_sch.iloc[t,2] =  model.i[t].value
  sys_op_sch.iloc[t,3] =  model.r[t].value

sys_op_sch = sys_op_sch.rename(columns={0:"Month",1:"electricity generation",2:"irrigation",3:"reservoir level"})
print(sys_op_sch)

    Month  electricity generation  irrigation  reservoir level
0     1.0                     1.0         0.0              6.0
1     2.0                     1.0         0.0              7.0
2     3.0                     1.0         0.0              9.0
3     4.0                     3.0         2.0             10.0
4     5.0                     3.0         2.0             10.0
5     6.0                     7.0         6.0              5.0
6     7.0                     7.0         6.0              0.0
7     8.0                     1.0         0.0              0.0
8     9.0                     2.0         1.0              0.0
9    10.0                     1.0         0.0              2.0
10   11.0                     1.0         0.0              4.0
11   12.0                     1.0         0.0              5.0


In [ ]:
model.dual=Suffix(direction=Suffix.IMPORT)

In [ ]:
#Solve Model
results = opt.solve(model)

print("Shadow Prices")
model.dual.pprint()

Shadow Prices
dual : Direction=IMPORT, Datatype=FLOAT
    Key             : Value
    constraints[10] :  -3.9
    constraints[11] :  -0.0
    constraints[12] :  -0.0
    constraints[13] :  -2.0
    constraints[14] :  -3.9
    constraints[15] :  -0.0
    constraints[16] :   0.3
    constraints[17] :  -2.2
    constraints[18] :  -4.2
    constraints[19] :  -0.0
     constraints[1] :  -2.3
    constraints[20] :  -0.0
    constraints[21] :  -2.2
    constraints[22] :  -4.2
    constraints[23] :  -0.0
    constraints[24] :  -0.0
    constraints[25] :  -2.5
    constraints[26] :  -4.2
    constraints[27] :   0.3
    constraints[28] :  -0.0
    constraints[29] :  -2.2
     constraints[2] :   3.9
    constraints[30] :  -4.1
    constraints[31] :  -0.0
    constraints[32] :  -0.0
    constraints[33] :  -1.8
    constraints[34] :  -3.6
    constraints[35] :  -0.0
    constraints[36] :  -0.0
    constraints[37] :  -1.9
    constraints[38] :  -3.6
    constraints[39] :  -0.0
     constraints[3] : 